* Name: Ishaan Thanekar
* USCID: 876551769
* Homework 7

## Q.1 

#### b)

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import rbf_kernel
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from sklearn.cluster import KMeans

In [2]:
df = pd.read_csv('HW7_Pr1_S24_training_data.csv')
df_t = pd.read_csv('HW7_Pr1_S24_test_data.csv')

In [3]:
df.head()

,x1,x2,y
0,0.028453,0.867236,-0.156244
1,-0.526357,0.754168,-1.452209
2,0.741746,-0.112823,-0.803381
3,0.711261,-0.415092,-2.281739
4,-0.009440,0.841013,-0.007648


In [4]:
df.isna().sum()

x1    0
x2    0
y     0
dtype: int64

In [5]:
df.shape

(1000, 3)

In [6]:
def data(dataframe):
    dataframe = dataframe.to_numpy()
    features = dataframe[:,:-1]
    labels = dataframe[:,-1]
    return features, labels

In [7]:
# baseline model function:

def baseline(features, labels, folds):
    kfolds = KFold(n_splits=folds, shuffle = True, random_state = 42)
    errors = []
    for x, y in kfolds.split(features):
        y_train, y_val = labels[x], labels[y]
        train_X, val_X = features[x], features[y]
        
        # defining the model: Linear Regression:
        model = LinearRegression()
        model.fit(train_X, y_train)
        preds = model.predict(val_X)
        # implementing RMSE according to the formula given to us:
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        errors.append(rmse)
        print(errors)
    # given is the average RMSE on the validation sets for baseline
    return np.mean(errors)

In [8]:
features, labels = data(df)
answerB = baseline(features, labels, folds=5)
print(f"Average RMSE on the validation sets is:  {answerB}")

[2.4119907651553927]
[2.4119907651553927, 2.466179520216617]
[2.4119907651553927, 2.466179520216617, 2.407442766333652]
[2.4119907651553927, 2.466179520216617, 2.407442766333652, 2.5226750780283713]
[2.4119907651553927, 2.466179520216617, 2.407442766333652, 2.5226750780283713, 2.3091403874066456]
Average RMSE on the validation sets is:  2.4234857034281356


#### c)

* Implementing the RBF Network as mentioned:

In [9]:
# Let's implement the rbf kernel from scratch as mentioned in equatiuon instead of importing because why not!
# Network:

# Here I first tested with my own rbf function but its optimal value varied when I used rbf_kernel
# so i used rbf_kernel a fucntion provided by sklearn which is optimized properly....

"""def rbf(x, u, gamma):
    d = euclidean_distances(x, u)
    # calculating the similarity score through rbf:
    return np.exp(-gamma * np.square(d))"""


def linear_model(features, labels):
    """I created this function as it was generating errors when I use Linear 
    Regression model directly inside my loop below, reference was taken after
    contacting the TA's and discussing with a few peers
    """
    model = LinearRegression()
    model.fit(features, labels)
    return model

# Now we create a model selection function for gammas using gamma d's
"""Note: So here and earlier the optimization of using kfolds.split was taken through a 
 Stack Overflow Post fetauring optimization in model_selection for best fit"""
 
def model_select_one(features, labels, gammas, folds = 5):
    errors = []
    valid_errors = []
    std_errors = []
    std_val_errors = []
    kfolds = KFold(n_splits=folds)
    for g in gammas:
        train_errors = []
        val_errors = []
        for x, y in kfolds.split(features):
            y_train, y_val = labels[x], labels[y]
            train_X, val_X = features[x], features[y]
            # defining feature space of phi-m as mentioned:
            # here i define phi-m as m
            m_tr = rbf_kernel(train_X, gamma=g)
            model = linear_model(m_tr, y_train)
            train_preds = model.predict(m_tr)
            
            # um's are the training of X i.e train_X
            m_val = rbf_kernel(val_X, train_X, g)
            val_preds = model.predict(m_val)
            # according to the formula
            rmse_tr = np.sqrt(mean_squared_error(y_train, train_preds))
            rmse_val = np.sqrt(mean_squared_error(y_val, val_preds))
            train_errors.append(rmse_tr)
            val_errors.append(rmse_val)
        
        errors.append(np.mean(train_errors)) 
        valid_errors.append(np.mean(val_errors))
        std_errors.append(np.std(train_errors))
        std_val_errors.append(np.std(val_errors))
         
    return errors, valid_errors, std_errors, std_val_errors       

In [10]:
# defining Gamma d as we got in the a) part: M/32 and resubstituting with delta_x:

# M = 800, as 1000 * 4/5: Kfold = 5
gamma_d = 800/32

#gamma_d = 1 / 8 * np.mean((np.max(features, axis = 0) - np.min(features, axis = 0) / features.shape[0]**(1/features.shape[1]))**2)

gamma_ = [0.01, 0.1, 1, 10, 100]
gammas = [gamma_d * i for i in gamma_]

tot_errors, valid_tot_error, std_train, std_val = model_select_one(features, labels, gammas)

gamma_star = gammas[np.argmin(valid_tot_error)]
rmse_star = np.min(valid_tot_error)
print("Gamma_d value: ",gamma_d,"\n")
print('Gamma optimal Value:', gamma_star,"\n")
print('RMSE optimal Value:', rmse_star,"\n")

print("-------------------PLOT-------------------")

plt.figure(figsize=(10,8))
plt.plot(gammas, tot_errors, marker = 'x', label = 'TrainRMSE')
plt.plot(gammas, valid_tot_error, marker = 'o', color = 'red', label = 'ValidRMSE')
# implement a log scale for gammas as mentioned in the question:
plt.xscale('log')
plt.xlabel('Gammas')
plt.ylabel('Average Root Mean Square Error(RMSE) over 5 folds')
plt.title('Average Training and Validation RMSE vs. Gamma over 5 folds')
plt.legend()
plt.show()

MemoryError: Unable to allocate 4.88 MiB for an array with shape (800, 800) and data type float64

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(gammas, tot_errors, marker = 'x', label = 'TrainRMSE')
plt.plot(gammas, valid_tot_error, marker = 'o', color = 'red', label = 'ValidRMSE')
print("-------------- Without Log Scale------------------")
plt.xlabel('Gammas without log scale')
plt.ylabel('Average Root Mean Square Error(RMSE) over 5 folds')
plt.title('Average Training and Validation RMSE vs. Gamma over 5 folds')
plt.legend()
plt.show()

In [ ]:
# The scale of Training errors are in lower end, hence i have also plotted them seperately for reference....
plt.plot(gammas, tot_errors, marker = 'x', label = 'TrainRMSE')
plt.xscale('log')

In [ ]:
table = PrettyTable()
fields = ['Gammas', 'Validation_Avg_RMSE', 'Std_of_RMSE']
table.field_names = fields

for iter in range(len(valid_tot_error)):
    row = [gamma_[iter], valid_tot_error[iter], std_val[iter]]
    table.add_row(row)

print(table)

#### d)

1. K= range(10,101,10)
2. Kfold
3. K loop, gamma

In [ ]:
def linear_model(features, labels):
    model = LinearRegression()
    model.fit(features, labels)
    return model

# Creating model for KMeans this function has a few things which I discussed with my peers and implemented...
 
def model_select_two(features, labels, folds = 5):
    
    K_errors = []
    K_valid_errors = []
    K_std_errors = []
    K_std_val_errors = []
    kfolds = KFold(n_splits=folds)
    k_range = range(10, 101, 10)
    optimal_rmse = float('inf')
    optimal_k = None
    optimal_gamma = None
    optimal_centers = None
    # setting K for range in 10, 100 with steps of 10
    for iter in k_range:
        # M / 32:
        gamma_d = iter/32 
        gammas = [gamma_d * 0.01, gamma_d * 0.1, gamma_d * 1, gamma_d * 10, gamma_d * 100]
        gamma_train, gamma_val = [],[]
        gamma_std_train, gamma_std_val = [],[]
        # the beginning....
        
        for g in gammas:
            train_errors = []
            val_errors = []
            for x, y in kfolds.split(features):
                y_train, y_val = labels[x], labels[y]
                train_X, val_X = features[x], features[y]
                
                # running KMeans multiple times as mentioned in question:
                # The solution varies everytime I run it for 4,5 times
                # So I am keeping 5 as constant now:
                for i in range(5):
                    kmeans = KMeans(n_clusters=iter, init='random').fit(train_X)
                    centers = kmeans.cluster_centers_
                    
                m_tr = rbf_kernel(train_X, Y= centers, gamma=g)
                model = linear_model(m_tr, y_train)
                train_preds = model.predict(m_tr)
                rmse_tr = np.sqrt(mean_squared_error(y_train, train_preds))
                
                # um's are the centers for training in KMeans
                m_val = rbf_kernel(val_X, Y=centers, gamma=g)
                val_preds = model.predict(m_val)
                # according to the formula
                rmse_val = np.sqrt(mean_squared_error(y_val, val_preds))
                train_errors.append(rmse_tr)
                val_errors.append(rmse_val)
            
            # The part to store centers reference was taken by discussing with peers and TA:
            mean_val_error = np.mean(val_errors)
            if mean_val_error < optimal_rmse:
                optimal_rmse = mean_val_error
                optimal_k = iter
                optimal_gamma = g
                optimal_centers = centers
            
            """gamma_star = gammas[np.argmin(valid_tot_error)]
            rmse_star = np.min(valid_tot_error)"""
              
            gamma_train.append(np.mean(train_errors))
            gamma_val.append(np.mean(val_errors))
            gamma_std_train.append(np.std(train_errors))
            gamma_std_val.append(np.std(val_errors))
              
        K_errors.append(gamma_train) 
        K_valid_errors.append(gamma_val)
        K_std_errors.append(gamma_std_train)
        K_std_val_errors.append(gamma_std_val)
    
    print(f"K Star: {optimal_k}, Gamma Star: {optimal_gamma}, RMSE star: {optimal_rmse}")
    return K_errors, K_valid_errors, K_std_errors, K_std_val_errors, optimal_centers, optimal_k, optimal_gamma, optimal_rmse 

In [ ]:
import warnings

In [ ]:
warnings.filterwarnings('ignore')
K_tot_errors, K_valid_tot_error, K_std_errors, K_std_val_errors, k_centers, k_star, k_gamma_star, rmse_star = model_select_two(features, labels)

In [ ]:
gamma_d = k_star/32
gamma = [0.01 * gamma_d, 0.1 * gamma_d, gamma_d, 10 * gamma_d, 100 * gamma_d]

plt.plot(gamma, K_tot_errors[-1], label = 'Training RMSE')
plt.plot(gamma, K_valid_tot_error[-1], label = 'Validation RMSE')
# Plotting over a log scale...
plt.xscale('log')
plt.xlabel('Gammas')
plt.ylabel('Average Root Mean Square Error(RMSE) over 5 folds ')
plt.title('Average Training and Validation RMSE vs. Gamma over 5 folds over KMeans')
plt.legend()
plt.show()

In [ ]:
plt.plot(gamma, K_tot_errors[-1], label = 'Training RMSE')
plt.plot(gamma, K_valid_tot_error[-1], label = 'Validation RMSE')
# Plotting over a log scale...
print("---------------Without Log Scale--------------------")
plt.xlabel('Gammas without log scale')
plt.ylabel('Average Root Mean Square Error(RMSE) over 5 folds ')
plt.title('Average Training and Validation RMSE vs. Gamma over 5 folds over KMeans')
plt.legend()
plt.show()

* Individual plots in LOG Scale:

In [ ]:
plt.plot(gamma, K_tot_errors[-1], label = 'Training RMSE')
plt.xscale('log')
plt.xlabel('Gammas')
plt.ylabel('Average Root Mean Square Error(RMSE) over 5 folds ')
plt.title('Average Training RMSE vs. Gamma over 5 folds over KMeans')
plt.legend()
plt.show()

In [ ]:
plt.plot(gamma, K_valid_tot_error[-1], label = 'Validation RMSE', color = 'red')
plt.xscale('log')
plt.xlabel('Gammas')
plt.ylabel('Average Root Mean Square Error(RMSE) over 5 folds ')
plt.title('Average Validation RMSE vs. Gamma over 5 folds over KMeans')
plt.legend()
plt.show()

* Plot with Optimal centers and data points

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(features[:, 0], features[:, 1], s=10, color='blue', label='Features')
plt.scatter(k_centers[:, 0], k_centers[:, 1], s=50, color='red', marker='x', label='Best Cluster Centers')
plt.title('Training Data Points and Best Cluster Centers')
plt.legend()
plt.show()

* Plot with Validation RMSE and it's standard deviation against all K's

In [ ]:
# since my gamma_ came to be 1 being multiplied by best K value of 90 and if I plot it
# It will be the third column i.e for gamma_ = 1
k_range = range(10, 101, 10)  
plt.figure(figsize=(10, 8))
plt.plot(list(k_range), [x[2] for x in K_valid_tot_error], label = "Validation RMSE")
plt.plot(list(k_range), [x[2] for x in K_std_val_errors], label = "STD of Validation RMSE")
#plt.errorbar(list(k_range), [x[2] for x in K_valid_tot_error], yerr=[x[2] for x in K_std_val_errors], fmt='-o', label='Validation RMSE')
plt.title("Validation RMSE and it's Standard Deviation vs. K for best gamma")
plt.xlabel('K = Number of Clusters')
plt.ylabel('Validation RMSE')
plt.legend()
plt.show()

In [ ]:
"""
Or if you want it in a single line with strokes for marking it it can also be done
with the help of errorbar this part was taken by referencing to errorbars 
in STACK-overflow
"""
plt.errorbar(list(k_range), [x[2] for x in K_valid_tot_error], yerr=[x[2] for x in K_std_val_errors], fmt='-o', label='Validation RMSE')
plt.title("Validation RMSE and it's Standard Deviation vs. K for best gamma")
plt.xlabel('K = Number of Clusters')
plt.ylabel('Validation RMSE')
plt.legend()
plt.show()

* Plotting Both seperately:

In [ ]:
plt.plot(list(k_range), [x[2] for x in K_valid_tot_error], label='Validation RMSE')
plt.title("Validation RMSE vs. K for best Gamma")
plt.xlabel('K = Number of Clusters')
plt.ylabel('Validation RMSE')
plt.legend()
plt.show()

In [ ]:
plt.plot(list(k_range), [x[2] for x in K_std_val_errors], label='Standard Deviation of RMSE', color = 'orange')
plt.title("Standard Deviation of  RMSE vs. K for best Gamma")
plt.xlabel('K = Number of Clusters')
plt.ylabel('Validation RMSE')
plt.legend()
plt.show()

In [ ]:
table = PrettyTable()
fields = ['KMeans Validation_Avg_RMSE', 'KMeans Std_of_RMSE']
table.field_names = fields

for iter in range(len(K_valid_tot_error[-1])):
    row = [K_valid_tot_error[-1][iter], K_std_val_errors[-1][iter]]
    table.add_row(row)
print(table)

#### e)

#### f)
* Testing on Testing Dataset

In [ ]:
features_t, labels_t = data(df_t)

* Testing Model part B:

In [ ]:
# Since we already train it using cross validation we just need to train it on 
# Linear Regression model and obtain TEST RMSE as directed in the question
model = LinearRegression().fit(features, labels)

test_preds = model.predict(features_t)
rmse_test = np.sqrt(mean_squared_error(labels_t, test_preds))
print(f"RMSE while Testing Model B: {rmse_test}")

* Testing for Model part C:

In [ ]:
# As we have Optimal Gamma as 0.25 obtained from the C part
training = rbf_kernel(features, gamma=0.25)
model = LinearRegression()
model.fit(training, labels)

# Testing:
testing = rbf_kernel(features_t, features, 0.25)
test_c_preds = model.predict(testing)
rmse_test_c = np.sqrt(mean_squared_error(labels_t, test_c_preds))
print(f"RMSE while Testing Model C: {rmse_test_c}")

* Testing for Model part D:

In [ ]:
# I will run KMeans again for 5 times as i did in my training as mentioned in the question:
# As we get in the D part that K star is 100 and Gamma star is 3.125 I will use that...
for i in range(5):
    kmeans = KMeans(n_clusters=100, init='random').fit(features)
    centers = kmeans.cluster_centers_

# Training with K star and Gamma Star
training_d = rbf_kernel(features, Y=centers, gamma=3.125)
testing_d = rbf_kernel(features_t, Y=centers, gamma=3.125)
model = LinearRegression()
model.fit(training_d, labels)

# Testing
predictions_test = model.predict(testing_d)
rmse_test_d = np.sqrt(mean_squared_error(labels_t, predictions_test))
print(f"RMSE while Testing Model D: {rmse_test_d}")

#### g)